In [1]:
import pandas as pd
import re

In [2]:
cat62 = pd.read_csv("data/cat-62.csv")

In [3]:

#cat62 = cat62[cat62["flightid"] != "Data final não disponível"]

In [4]:


cat62['dt_radar'] = pd.to_datetime(cat62['dt_radar'], unit='ms')

In [5]:
bimtra = pd.read_csv("data/bimtra.csv")

In [6]:

bimtra['dt_dep'] = pd.to_datetime(bimtra['dt_dep'], unit='ms')
bimtra['dt_arr'] = pd.to_datetime(bimtra['dt_arr'], unit='ms')

In [7]:
bimtra["target"] = (bimtra["dt_arr"] - bimtra["dt_dep"]) // pd.Timedelta('1s')

In [8]:
bimtra["linha"] = bimtra["origem"] + bimtra["destino"]

In [9]:
bimtra = bimtra.drop_duplicates(subset='flightid')

In [10]:
cat62 = cat62.merge(bimtra, on="flightid", how="left")

In [11]:
cat62["linha"] = cat62["origem"] + cat62["destino"]

In [12]:
cat62.sort_values(by=['flightid', 'dt_radar'], inplace=True)

In [17]:
from math import radians, sin, cos, sqrt, atan2, degrees

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # raio da Terra em km

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [14]:


cat62["lat"] = cat62["lat"].apply(degrees)
cat62["lon"] = cat62["lon"].apply(degrees)

In [15]:
resumo_voo = cat62.groupby('flightid').agg({
    'dt_radar': lambda x: (x.max() - x.min()).total_seconds(),
    'lat': ['first', 'last'],
    'lon': ['first', 'last'],
    'flightlevel': ['mean', 'min', 'max', 'std'],
    'speed': ['mean', 'min', 'max', 'std'],
    'linha': 'first'
})


NameError: name 'haversine' is not defined

In [18]:
resumo_voo['distancia'] = resumo_voo.apply(lambda row: haversine(row[('lat', 'first')], row[('lon', 'first')], 
                                                           row[('lat', 'last')], row[('lon', 'last')]), axis=1)


In [20]:
# Aplicar a função join às colunas do DataFrame
resumo_voo.reset_index(inplace=True)


In [27]:
resumo_voo

,flightid_,dt_radar_<lambda>,lat_first,lat_last,lon_first,lon_last,flightlevel_mean,flightlevel_min,flightlevel_max,flightlevel_std,speed_mean,speed_min,speed_max,speed_std,linha_first,distancia_
0,0000f76a51b99be3f181ea2b201b34ef,6543.0,-12.898787,-22.453113,-38.277699,-46.23334,308.194737,14.0,340.0,70.927036,409.636842,170.0,440.0,42.68997,SBSVSBGR,1355.300518
1,00010bb2c49076d56a9815122fefc8aa,4083.0,-19.644736,-23.626155,-43.958119,-46.656405,214.356522,25.0,360.0,115.735496,312.017391,106.0,424.0,94.817274,SBCFSBSP,523.193228
2,00010ed6d09c78b5a37f4df13f524f56,7740.0,-13.035493,-23.623237,-38.282166,-46.658587,318.598086,24.0,380.0,106.012886,389.07177,145.0,436.0,58.582779,SBSVSBSP,1471.285722
3,000143522a56fa245543e477c74fd908,10019.0,-23.609967,-8.13566,-46.567788,-34.923127,298.994382,1.0,370.0,118.987703,416.511236,123.0,477.0,83.686207,SBSPSBRF,2121.248922
4,00014fa436546dfdc7a67919e9e1ea47,4924.0,-19.641217,-27.365809,-43.961015,-48.217413,281.933824,26.0,360.0,100.075024,393.022059,142.0,449.0,63.645601,SBCFSBFL,962.125338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278094,fffeba6342187a9ee0caff2526e4ff0e,4563.0,-23.578866,-29.996528,-46.689515,-51.181826,263.0,1.0,380.0,129.199031,364.925926,122.0,433.0,79.087701,SBSPSBPA,841.245187
278095,ffff413d10cfbb298a9411855e80a0f0,6602.0,-22.874854,-29.994711,-43.126958,-51.177427,305.910995,5.0,380.0,117.219396,335.778351,110.0,419.0,58.055247,SBRJSBPA,1125.897592
278096,ffff62189abb326d9f18ff1412e5de8e,3782.0,-19.63071,-23.437804,-43.970909,-46.493859,187.427273,26.0,360.0,126.294796,317.663636,142.0,421.0,92.550435,None,497.260753
278097,ffffd0e5ce497de7ddf3d34d19b9ce3a,2581.0,-23.590285,-22.939142,-46.671998,-43.155191,144.435897,3.0,270.0,93.35362,307.051282,120.0,372.0,73.091812,SBSPSBRJ,366.466603


In [22]:
resumo_voo = resumo_voo.T.reset_index().T

In [24]:
arr = []
for i in range(0, len(resumo_voo.iloc[0].values)):
    arr.append(resumo_voo.iloc[0].values[i] + "_" + resumo_voo.iloc[1].values[i])

In [25]:
resumo_voo = resumo_voo.rename(columns=dict(zip(resumo_voo.columns, arr)))


In [26]:
resumo_voo = resumo_voo.iloc[2:]

In [28]:
resumo = resumo_voo.groupby('linha_first').agg({
    'flightid_': 'count',
    'dt_radar_<lambda>': ['mean', 'min', 'max', 'std'],
    'flightlevel_mean': 'mean',
    'flightlevel_max': 'mean',
    'distancia_': 'mean',
    'flightlevel_std': ['std', 'mean'],
    'speed_std': ['mean','std']
})

# Exibir o resumo
print(resumo)

            flightid_ dt_radar_<lambda>                                \
                count              mean     min      max          std   
linha_first                                                             
SBBRSBCF         2644       3210.663767     0.0   8277.0   506.981575   
SBBRSBCT          959       5524.459854     4.0   8166.0   602.814715   
SBBRSBFL          315       6614.263492     4.0  14041.0   824.447364   
SBBRSBGL          639       4499.599374   362.0   7138.0   539.795848   
SBBRSBGR         3124       4384.066261   238.0  15064.0   636.873657   
...               ...               ...     ...      ...          ...   
SBSVSBKP         1379       7368.569978   121.0  11225.0   761.985305   
SBSVSBPA           79      11771.683544  1683.0  13623.0  1455.223285   
SBSVSBRF         1491        3784.31053     0.0   9182.0   526.762331   
SBSVSBRJ         1630       6706.908589   843.0  10924.0   630.301570   
SBSVSBSP         3519       7789.326229   305.0  15

In [29]:
resumo.reset_index(inplace=True)
resumo.to_csv("data/resumo_voo.csv", index=False)

In [2]:
import pandas as pd
resumo = pd.read_csv("data/resumo_voo.csv")
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,distancia_,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,575.6958980433018,15.414484379289922,113.11099040127081,96.26012004571069,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,1061.908684010797,13.184672048521254,113.58039360634044,84.08358708763086,13.509180931157234
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,1282.3324062624968,15.232888859932347,115.89589711595092,79.03449466219763,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,885.994544233408,13.252080132619195,122.03779859280792,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,1426.4684886193365,17.341259072228155,102.26689170812047,68.5258508294501,16.681647582814303
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,2277.3825023041554,8.22824879734405,99.87921093160097,62.36285594137942,8.362795726130804
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,643.117568170169,15.707041969914366,127.34337283196844,97.04344098509462,14.56471447373063
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,1225.918106541467,12.702016763699218,123.87140594500069,82.84562463100339,13.215119734561947


In [22]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,flightlevel_mean,flightlevel_max,flightlevel_std,flightlevel_std.1,speed_std,speed_std.1
0,NaN,count,mean,min,max,std,mean,mean,std,mean,mean,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,223.23009635516672,352.7874432677761,15.414484379289918,113.11099040127084,96.26012004571065,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,281.071385902067,367.9061522419187,13.184672048521252,113.58039360634044,84.08358708763087,13.50918093115723
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,291.94466776671055,371.6634920634921,15.232888859932348,115.89589711595086,79.03449466219764,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,259.66979013256446,367.9154929577465,13.252080132619195,122.03779859280796,92.35050625485451,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,305.3446206649822,369.22770123277735,17.341259072228155,102.26689170812047,68.52585082945009,16.681647582814314
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,324.43509942690906,375.1898734177215,8.228248797344037,99.87921093160097,62.36285594137942,8.362795726130809
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,226.64509710011185,362.5083836351442,15.70704196991436,127.34337283196848,97.04344098509465,14.564714473730632
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,291.5118729723326,378.08895705521473,12.702016763699223,123.87140594500069,82.84562463100339,13.215119734561949


In [35]:
esperas = pd.read_csv("data/esperas.csv")
esperas["dt_dep"] = pd.to_datetime(esperas["hora"], unit='ms')
esperas["hora"] = pd.to_datetime(esperas["hora"], unit='ms')
esperas["diaSemana"] = esperas["hora"].dt.dayofweek

esperas["hora"] = esperas["hora"].dt.hour

In [24]:
esperas_hora = esperas.groupby(["aero", "hora"]).agg({"esperas": ["mean", "sum"]})
esperas_semana = esperas.groupby(["aero", "diaSemana"]).agg({"esperas": ["mean", "sum"]})

In [25]:

esperas_aero = esperas.groupby(["aero"]).agg({"esperas": ["mean", "sum"]})

In [36]:
esperas

,esperas,hora,aero,dt_dep,diaSemana
0,0,0,SBBR,2022-06-01 00:00:00,2
1,0,1,SBBR,2022-06-01 01:00:00,2
2,0,2,SBBR,2022-06-01 02:00:00,2
3,0,3,SBBR,2022-06-01 03:00:00,2
4,0,4,SBBR,2022-06-01 04:00:00,2
...,...,...,...,...,...
93307,0,19,SBSV,2023-04-15 19:00:00,5
93308,0,20,SBSV,2023-04-15 20:00:00,5
93309,0,21,SBSV,2023-04-15 21:00:00,5
93310,0,22,SBSV,2023-04-15 22:00:00,5


In [21]:
bimtra["diaSemana"] = bimtra["dt_dep"].dt.dayofweek

bimtra["hora"] = bimtra["dt_dep"].dt.hour

In [ ]:
bimtra['HoraDataDest'] = bimtra['dt_arr'].dt.strftime('%Y-%m-%d %H')

In [42]:
bimtra['HoraData'] = bimtra['dt_dep'].dt.strftime('%Y-%m-%d %H')
esperas['HoraData'] = esperas['dt_dep'].dt.strftime('%Y-%m-%d %H')

# Mesclar os dataframes usando a coluna "HoraData"
df_merge = pd.merge(bimtra, esperas, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['esperas'] = df_merge['esperas'].fillna(0)
df_merge = df_merge.groupby("flightid").agg({"esperas": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

                                 flightid origem destino            dt_dep_x  \
0        fcb2bf90345705318213ae1307c0f901   SBKP    SBRJ 2022-06-01 00:44:57   
1        fcb2bf90345705318213ae1307c0f901   SBKP    SBRJ 2022-06-01 00:44:57   
2        fcb2bf90345705318213ae1307c0f901   SBKP    SBRJ 2022-06-01 00:44:57   
3        fcb2bf90345705318213ae1307c0f901   SBKP    SBRJ 2022-06-01 00:44:57   
4        fcb2bf90345705318213ae1307c0f901   SBKP    SBRJ 2022-06-01 00:44:57   
...                                   ...    ...     ...                 ...   
3448039  d36199f6d9a3febc09113013e253739d   SBPA    SBRJ 2023-04-15 17:59:42   
3448040  d36199f6d9a3febc09113013e253739d   SBPA    SBRJ 2023-04-15 17:59:42   
3448041  d36199f6d9a3febc09113013e253739d   SBPA    SBRJ 2023-04-15 17:59:42   
3448042  d36199f6d9a3febc09113013e253739d   SBPA    SBRJ 2023-04-15 17:59:42   
3448043  d36199f6d9a3febc09113013e253739d   SBPA    SBRJ 2023-04-15 17:59:42   

                     dt_arr  target    

,esperas
flightid,
0000f76a51b99be3f181ea2b201b34ef,1
00010bb2c49076d56a9815122fefc8aa,1
00010ed6d09c78b5a37f4df13f524f56,0
000143522a56fa245543e477c74fd908,1
00014fa436546dfdc7a67919e9e1ea47,3
...,...
fffea9a5cfeeb8f24e738cf60a1d5f2b,3
fffeba6342187a9ee0caff2526e4ff0e,0
ffff413d10cfbb298a9411855e80a0f0,0


In [45]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraData,esperas
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 00,0
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 00,0
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 00,0
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 00,0
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 01,1
...,...,...,...,...,...,...,...,...,...,...,...
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 12,0
278885,82ab2350af0301d57fd2041b7504f919,SBGR,SBRJ,2023-04-15 13:14:51,2023-04-15 13:54:44,2393,SBGRSBRJ,5,13,2023-04-15 13,0
278886,25fe7303ad56ca0f9976b8243546305b,SBSP,SBRJ,2023-04-15 17:11:23,2023-04-15 18:11:42,3619,SBSPSBRJ,5,17,2023-04-15 17,4
278887,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,2023-04-15 17,8


In [25]:
bimtra.merge(esperas, left_on=["origem", "hora", "diaSemana"], right_on=["aero", "hora", "diaSemana"], how="left")


,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,esperas,aero
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,0,SBKP
1,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,0,SBKP
2,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,0,SBKP
3,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,0,SBKP
4,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,0,SBKP
...,...,...,...,...,...,...,...,...,...,...,...
12916186,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,0,SBPA
12916187,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,0,SBPA
12916188,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,0,SBPA
12916189,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,0,SBPA


In [29]:
esperas_hora.columns = esperas_hora.columns.droplevel(0)

In [30]:
esperas_hora.reset_index(inplace=True)

In [31]:
esperas_semana.columns = esperas_semana.columns.droplevel(0)
esperas_semana.reset_index(inplace=True)

In [32]:
esperas_aero.columns = esperas_aero.columns.droplevel(0)
esperas_aero.reset_index(inplace=True)

In [33]:
esperas_semana

,aero,diaSemana,mean,sum
0,SBBR,0,0.009058,10
1,SBBR,1,0.019022,21
2,SBBR,2,0.034420,38
3,SBBR,3,0.022163,25
4,SBBR,4,0.069149,78
...,...,...,...,...
79,SBSV,2,0.011775,13
80,SBSV,3,0.015071,17
81,SBSV,4,0.006206,7
82,SBSV,5,0.003623,4


In [34]:
bimtra = bimtra.merge(esperas_hora, right_on=["aero", "hora"], left_on=["origem", "hora"] , how="left")
bimtra = bimtra.merge(esperas_semana, right_on=["aero", "diaSemana"], left_on=["origem", "diaSemana"] , how="left")

In [35]:
bimtra = bimtra.merge(esperas_aero, right_on=["aero"], left_on=["origem"] , how="left")

In [36]:
bimtra["origem"].unique()

array(['SBKP', 'SBGR', 'SBBR', 'SBCF', 'SBSP', 'SBRJ', 'SBFL', 'SBCT',
       'SBRF', 'SBPA', 'SBSV', 'SBGL'], dtype=object)

In [37]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,aero_x,mean_x,sum_x,aero_y,mean_y,sum_y,aero,mean,sum
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,SBKP,0.694444,225,SBKP,0.221014,244,SBKP,0.191872,1492
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,SBGR,0.987654,320,SBGR,0.254529,281,SBGR,0.255401,1986
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,SBBR,0.000000,0,SBBR,0.034420,38,SBBR,0.024434,190
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,SBKP,0.694444,225,SBKP,0.221014,244,SBKP,0.191872,1492
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,SBCF,0.055556,18,SBCF,0.173007,191,SBCF,0.150463,1170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,SBGR,0.123457,40,SBGR,0.167572,185,SBGR,0.255401,1986
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,SBBR,0.074074,24,SBBR,0.004529,5,SBBR,0.024434,190
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,SBFL,0.033951,11,SBFL,0.012681,14,SBFL,0.015947,124
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,SBSV,0.000000,0,SBSV,0.003623,4,SBSV,0.009259,72


In [11]:
import pandas as pd
metar = pd.read_csv("data/metar.csv")

In [2]:
metar

,hora,metar,aero
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV


In [101]:
import pandas as pd
from metar.Metar import Metar

# Função para parsear METAR
def parse_metar(metar):
    metar = metar.replace('METAF', 'METAR') 
    try:
        metar_obj = Metar(metar)
        temperature = metar_obj.temp.value() if metar_obj.temp else None
        dewpoint = metar_obj.dewpt.value() if metar_obj.dewpt else None
        wind_speed = metar_obj.wind_speed.value() if metar_obj.wind_speed else None
        wind_direction = metar_obj.wind_dir.value() if metar_obj.wind_dir else None
        visibility = metar_obj.vis.value() if metar_obj.vis else None
        pressure = metar_obj.press.value() if metar_obj.press else None
        weather = ', '.join([str(c) for c in metar_obj.weather]) if metar_obj.weather else None
        sky = ', '.join([str(c) for c in metar_obj.sky]) if metar_obj.sky else None
        return pd.Series([temperature, dewpoint, wind_speed, wind_direction, visibility, pressure, weather, sky])
    except Exception as e:
        #print(str(e))
        return pd.Series([None, None, None, None, None, None, None, None])

# Aplicar parse sobre METAR para cada linha
metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metar['metar'].apply(parse_metar)


Unparsed groups in body '2045Z' while processing 'SPECI SBRF 2045Z 06009KT 030V100 2000 R18///// R36///// -RA SCT010 BKN015 BKN050 24/20 Q1011='


/tmp/ipykernel_594411/1085531158.py:23: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metar[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metar['metar'].apply(parse_metar)


In [102]:
metar

,hora,metar,aero,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento,Visibilidade,Pressão,Tempo,Céu,HoraData
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR,21.0,8.0,2.0,70.0,10000.0,1018.0,NaN,NaN,2022-06-01 00
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR,20.0,9.0,2.0,100.0,10000.0,1019.0,NaN,NaN,2022-06-01 01
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,0.0,0.0,10000.0,1019.0,NaN,NaN,2022-06-01 02
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR,16.0,9.0,2.0,270.0,10000.0,1019.0,NaN,NaN,2022-06-01 03
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR,14.0,9.0,0.0,0.0,10000.0,1018.0,NaN,NaN,2022-06-01 04
...,...,...,...,...,...,...,...,...,...,...,...,...
99617,1681585200000,METAR SBSV 151900Z 08008KT 050V110 9999 FEW030...,SBSV,29.0,23.0,8.0,80.0,10000.0,1011.0,None,"('FEW', <metar.Datatypes.distance object at 0x...",2023-04-15 19
99618,1681588800000,METAR SBSV 152000Z 09007KT 060V120 CAVOK 29/23...,SBSV,29.0,23.0,7.0,90.0,10000.0,1011.0,None,NaN,2023-04-15 20
99619,1681592400000,METAR SBSV 152100Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0,10000.0,1011.0,None,NaN,2023-04-15 21
99620,1681596000000,METAR SBSV 152200Z 08007KT CAVOK 28/22 Q1011=,SBSV,28.0,22.0,7.0,80.0,10000.0,1011.0,None,NaN,2023-04-15 22


Unparsed groups in body '010/04' while processing 'METAR SBGR 132100Z 01004KT CAVOK 010/04 Q1017='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132200Z  11003KT CAVOK 010/03 Q1022='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132300Z  14003KT CAVOK 010/03 Q1023='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140000Z  14006KT 0450 FG     OVC018   010/09 Q1018='
Unparsed groups in body '010/10' while processing 'METAR SBGR 140300Z 06005KT 0600 FG     OVC033   010/10 Q1017='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140400Z  10005KT 0800 FG     OVC033   010/09 Q1020='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140500Z  11005KT 0700 FG     OVC033   010/09 Q1021='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150400Z 09004KT 1000     BR OVC013   010/09 Q1022='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150800Z 09004KT 0700 FG     OVC013   010/09 Q1020='
Unparsed group

/tmp/ipykernel_594411/837609518.py:4: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf['metaf'].apply(parse_metar)


Unparsed groups in body '010/04' while processing 'METAR SBGR 132100Z 01004KT CAVOK 010/04 Q1017='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132200Z 11003KT CAVOK 010/03 Q1022='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132300Z 14003KT CAVOK 010/03 Q1023='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140000Z 14006KT 0450 FG OVC018 010/09 Q1018='
Unparsed groups in body '010/10' while processing 'METAR SBGR 140300Z 06005KT 0600 FG OVC033 010/10 Q1017='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140400Z 10005KT 0800 FG OVC033 010/09 Q1020='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140500Z 11005KT 0700 FG OVC033 010/09 Q1021='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150400Z 09004KT 1000 BR OVC013 010/09 Q1022='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150800Z 09004KT 0700 FG OVC013 010/09 Q1020='
Unparsed groups in body '010/05' while processing 'META

/tmp/ipykernel_594411/837609518.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metaf['metaf'].apply(parse_metar)


,hora,metaf,aero,Temperatura,Ponto de Orvalho,Velocidade do Vento,Direção do Vento,Visibilidade,Pressão,Tempo,Céu
0,1654041600000,METAF SBGL 010000Z 25002KT 3000 BR OVC033...,SBGL,23.0,21.0,2.0,250.0,3000.0,1016.0,"('', None, '', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."
1,1654045200000,METAF SBGL 010100Z 02001KT 2000 BR OVC011 ...,SBGL,22.0,21.0,1.0,20.0,2000.0,1017.0,"('', None, '', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."
2,1654048800000,METAF SBGL 010200Z 09000KT 2000 BR OVC011 ...,SBGL,22.0,21.0,0.0,90.0,2000.0,1018.0,"('', None, '', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."
3,1654052400000,METAF SBGL 010300Z 09002KT 1000 RABR OVC033...,SBGL,22.0,21.0,2.0,90.0,1000.0,1017.0,"('', None, 'RA', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."
4,1654056000000,METAF SBGL 010400Z 07003KT 1000 RABR OVC033...,SBGL,22.0,21.0,3.0,70.0,1000.0,1016.0,"('', None, 'RA', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."
...,...,...,...,...,...,...,...,...,...,...,...
37837,1681585200000,METAF SBSP 151900Z 14009KT 9999 OVC020 ...,SBSP,22.0,16.0,9.0,140.0,10000.0,1014.0,None,"('OVC', <metar.Datatypes.distance object at 0x..."
37838,1681588800000,METAF SBSP 152000Z 14009KT 9999 OVC015 ...,SBSP,21.0,17.0,9.0,140.0,10000.0,1015.0,None,"('OVC', <metar.Datatypes.distance object at 0x..."
37839,1681592400000,METAF SBSP 152100Z 13009KT 6000 OVC011 ...,SBSP,20.0,17.0,9.0,130.0,6000.0,1015.0,None,"('OVC', <metar.Datatypes.distance object at 0x..."
37840,1681596000000,METAF SBSP 152200Z 12008KT 4000 BR OVC033...,SBSP,20.0,17.0,8.0,120.0,4000.0,1015.0,"('', None, '', 'BR', None)","('OVC', <metar.Datatypes.distance object at 0x..."


In [95]:
metar.iloc[0]["metar"]

'METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018='

In [75]:

metar['HoraData'] = pd.to_datetime(metar['hora'], unit='ms').dt.strftime('%Y-%m-%d %H')
# Mesclar os dataframes usando a coluna "HoraData"



# Preencher NaN na coluna de espera com 0 (sem espera)


In [84]:
# Agrupando por flight_id
df_merge = pd.merge(bimtra, metar, left_on=['origem', 'HoraData'], right_on=['aero', 'HoraData'], how='left')

grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.reset_index(inplace=True)
stats_df.columns = [''.join(col) for col in stats_df.columns]
bimtra = bimtra.merge(stats_df, on="flightid", how="left")

In [85]:
# Agrupando por flight_id
df_merge = pd.merge(bimtra, metar, left_on=['destino', 'HoraDataDest'], right_on=['aero', 'HoraData'], how='left')

grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.reset_index(inplace=True)
stats_df.columns = [''.join(col) for col in stats_df.columns]
bimtra = bimtra.merge(stats_df, on="flightid", how="left")

,flightid,Temperaturamean,Temperaturamin,Temperaturamax,Temperaturastd,Ponto de Orvalhomean,Ponto de Orvalhomin,Ponto de Orvalhomax,Ponto de Orvalhostd,Velocidade do Ventomean,...,Direção do Ventomax,Direção do Ventostd,Visibilidademean,Visibilidademin,Visibilidademax,Visibilidadestd,Pressãomean,Pressãomin,Pressãomax,Pressãostd
0,0000f76a51b99be3f181ea2b201b34ef,25.0,25.0,25.0,NaN,22.0,22.0,22.0,NaN,0.0,...,0.0,NaN,10000.0,10000.0,10000.0,NaN,1012.0,1012.0,1012.0,NaN
1,00010bb2c49076d56a9815122fefc8aa,21.5,21.0,22.0,0.707107,20.0,20.0,20.0,0.0,4.0,...,310.0,28.284271,5000.0,4000.0,6000.0,1414.213562,1010.5,1010.0,1011.0,0.707107
2,00010ed6d09c78b5a37f4df13f524f56,28.0,28.0,28.0,NaN,24.0,24.0,24.0,NaN,5.0,...,60.0,NaN,9000.0,9000.0,9000.0,NaN,1014.0,1014.0,1014.0,NaN
3,000143522a56fa245543e477c74fd908,23.0,23.0,23.0,NaN,18.0,18.0,18.0,NaN,13.0,...,340.0,NaN,10000.0,10000.0,10000.0,NaN,1011.0,1011.0,1011.0,NaN
4,00014fa436546dfdc7a67919e9e1ea47,22.0,22.0,22.0,NaN,20.0,20.0,20.0,NaN,2.0,...,130.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,fffea9a5cfeeb8f24e738cf60a1d5f2b,29.0,29.0,29.0,NaN,13.0,13.0,13.0,NaN,6.0,...,230.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
262412,fffeba6342187a9ee0caff2526e4ff0e,26.0,26.0,26.0,0.000000,17.0,17.0,17.0,0.0,9.0,...,340.0,0.000000,10000.0,10000.0,10000.0,0.000000,1017.0,1017.0,1017.0,0.000000
262413,ffff413d10cfbb298a9411855e80a0f0,21.0,21.0,21.0,NaN,14.0,14.0,14.0,NaN,6.0,...,350.0,NaN,10000.0,10000.0,10000.0,NaN,1020.0,1020.0,1020.0,NaN
262414,ffffd0e5ce497de7ddf3d34d19b9ce3a,18.0,18.0,18.0,NaN,10.0,10.0,10.0,NaN,5.0,...,40.0,NaN,10000.0,10000.0,10000.0,NaN,1025.0,1025.0,1025.0,NaN


In [87]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraData,...,Direção do Ventomax,Direção do Ventostd,Visibilidademean,Visibilidademin,Visibilidademax,Visibilidadestd,Pressãomean,Pressãomin,Pressãomax,Pressãostd
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 00,...,120.0,NaN,10000.0,10000.0,10000.0,NaN,1017.0,1017.0,1017.0,NaN
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 00,...,130.0,NaN,10000.0,10000.0,10000.0,NaN,1019.0,1019.0,1019.0,NaN
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 00,...,70.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 00,...,120.0,NaN,10000.0,10000.0,10000.0,NaN,1017.0,1017.0,1017.0,NaN
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 01,...,100.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 12,...,70.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
278885,82ab2350af0301d57fd2041b7504f919,SBGR,SBRJ,2023-04-15 13:14:51,2023-04-15 13:54:44,2393,SBGRSBRJ,5,13,2023-04-15 13,...,200.0,NaN,8000.0,8000.0,8000.0,NaN,1019.0,1019.0,1019.0,NaN
278886,25fe7303ad56ca0f9976b8243546305b,SBSP,SBRJ,2023-04-15 17:11:23,2023-04-15 18:11:42,3619,SBSPSBRJ,5,17,2023-04-15 17,...,150.0,0.0,10000.0,10000.0,10000.0,0.0,1019.0,1019.0,1019.0,0.0
278887,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,2023-04-15 17,...,130.0,0.0,10000.0,10000.0,10000.0,0.0,1016.0,1016.0,1016.0,0.0


In [108]:
import pandas as pd

metaf = pd.read_csv("data/metaf.csv")
metaf['metaf'].apply(parse_metar)
metaf['metaf'] = metaf['metaf'].apply(lambda x: ' '.join(x.split()))

metaf[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metaf['metaf'].apply(parse_metar)

metaf['HoraData'] = pd.to_datetime(metaf['hora'], unit='ms').dt.strftime('%Y-%m-%d %H')
# Mesclar os dataframes usando a coluna "HoraData"

df_merge = pd.merge(bimtra, metar, left_on=['origem', 'HoraData'], right_on=['aero', 'HoraData'], how='left')
# Agrupando por flight_id
grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.columns = [''.join(col) for col in stats_df.columns]
stats_df.reset_index(inplace=True)

bimtra = bimtra.merge(stats_df, on="flightid", how="left")
# Preencher NaN na coluna de espera com 0 (sem espera)
bimtra

Unparsed groups in body '010/04' while processing 'METAR SBGR 132100Z 01004KT CAVOK 010/04 Q1017='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132200Z  11003KT CAVOK 010/03 Q1022='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132300Z  14003KT CAVOK 010/03 Q1023='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140000Z  14006KT 0450 FG     OVC018   010/09 Q1018='
Unparsed groups in body '010/10' while processing 'METAR SBGR 140300Z 06005KT 0600 FG     OVC033   010/10 Q1017='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140400Z  10005KT 0800 FG     OVC033   010/09 Q1020='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140500Z  11005KT 0700 FG     OVC033   010/09 Q1021='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150400Z 09004KT 1000     BR OVC013   010/09 Q1022='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150800Z 09004KT 0700 FG     OVC013   010/09 Q1020='
Unparsed group

/tmp/ipykernel_594411/1710600021.py:4: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf['metaf'].apply(parse_metar)


Unparsed groups in body '010/04' while processing 'METAR SBGR 132100Z 01004KT CAVOK 010/04 Q1017='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132200Z 11003KT CAVOK 010/03 Q1022='
Unparsed groups in body '010/03' while processing 'METAR SBGR 132300Z 14003KT CAVOK 010/03 Q1023='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140000Z 14006KT 0450 FG OVC018 010/09 Q1018='
Unparsed groups in body '010/10' while processing 'METAR SBGR 140300Z 06005KT 0600 FG OVC033 010/10 Q1017='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140400Z 10005KT 0800 FG OVC033 010/09 Q1020='
Unparsed groups in body '010/09' while processing 'METAR SBGR 140500Z 11005KT 0700 FG OVC033 010/09 Q1021='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150400Z 09004KT 1000 BR OVC013 010/09 Q1022='
Unparsed groups in body '010/09' while processing 'METAR SBGR 150800Z 09004KT 0700 FG OVC013 010/09 Q1020='
Unparsed groups in body '010/05' while processing 'META

/tmp/ipykernel_594411/1710600021.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  metaf[['Temperatura', 'Ponto de Orvalho', 'Velocidade do Vento', 'Direção do Vento', 'Visibilidade', 'Pressão', 'Tempo', 'Céu']] = metaf['metaf'].apply(parse_metar)


,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraData,...,Direção do Vento_max_y,Direção do Vento_std_y,Visibilidade_mean_y,Visibilidade_min_y,Visibilidade_max_y,Visibilidade_std_y,Pressão_mean_y,Pressão_min_y,Pressão_max_y,Pressão_std_y
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 00,...,120.0,NaN,10000.0,10000.0,10000.0,NaN,1017.0,1017.0,1017.0,NaN
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 00,...,130.0,NaN,10000.0,10000.0,10000.0,NaN,1019.0,1019.0,1019.0,NaN
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 00,...,70.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 00,...,120.0,NaN,10000.0,10000.0,10000.0,NaN,1017.0,1017.0,1017.0,NaN
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 01,...,100.0,NaN,10000.0,10000.0,10000.0,NaN,1018.0,1018.0,1018.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 12,...,70.0,NaN,10000.0,10000.0,10000.0,NaN,1014.0,1014.0,1014.0,NaN
278885,82ab2350af0301d57fd2041b7504f919,SBGR,SBRJ,2023-04-15 13:14:51,2023-04-15 13:54:44,2393,SBGRSBRJ,5,13,2023-04-15 13,...,200.0,NaN,8000.0,8000.0,8000.0,NaN,1019.0,1019.0,1019.0,NaN
278886,25fe7303ad56ca0f9976b8243546305b,SBSP,SBRJ,2023-04-15 17:11:23,2023-04-15 18:11:42,3619,SBSPSBRJ,5,17,2023-04-15 17,...,150.0,0.0,10000.0,10000.0,10000.0,0.0,1019.0,1019.0,1019.0,0.0
278887,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,2023-04-15 17,...,130.0,0.0,10000.0,10000.0,10000.0,0.0,1016.0,1016.0,1016.0,0.0


In [ ]:

df_merge = pd.merge(bimtra, metar, left_on=['destino', 'HoraDataDest'], right_on=['aero', 'HoraData'], how='left')
# Agrupando por flight_id
grouped_df = df_merge.groupby('flightid')

# Calculando as estatísticas
stats_df = grouped_df.agg({
    'Temperatura': ['mean', 'min', 'max', 'std'],
    'Ponto de Orvalho': ['mean', 'min', 'max', 'std'],
    'Velocidade do Vento': ['mean', 'min', 'max', 'std'],
    'Direção do Vento': ['mean', 'min', 'max', 'std'],
    'Visibilidade': ['mean', 'min', 'max', 'std'],
    'Pressão': ['mean', 'min', 'max', 'std']
})
stats_df.columns = [''.join(col) for col in stats_df.columns]
stats_df.reset_index(inplace=True)

bimtra = bimtra.merge(stats_df, on="flightid", how="left")
# Preencher NaN na coluna de espera com 0 (sem espera)
bimtra

In [46]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [47]:
tcprev["hora"] = pd.to_datetime(tcprev["hora"], unit='ms')

In [48]:
tcprev

,hora,troca,aero
0,2022-06-01 00:00:00,0,BR
1,2022-06-01 01:00:00,0,BR
2,2022-06-01 02:00:00,0,BR
3,2022-06-01 03:00:00,0,BR
4,2022-06-01 04:00:00,0,BR
...,...,...,...
93307,2023-04-15 19:00:00,1,SV
93308,2023-04-15 20:00:00,1,SV
93309,2023-04-15 21:00:00,1,SV
93310,2023-04-15 22:00:00,0,SV


In [ ]:
# Mesclar os dataframes usando a coluna "HoraData"
tcprev['HoraData'] = tcprev['hora'].dt.strftime('%Y-%m-%d %H')
df_merge = pd.merge(bimtra, tcprev, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['troca'] = df_merge['troca'].fillna(0)
df_merge = df_merge.groupby("flightid").agg({"troca": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

In [40]:
tcprev["diaSemana"] = tcprev["hora"].dt.dayofweek

tcprev["hora"] = tcprev["hora"].dt.hour
tcprev_hora = tcprev.groupby(["aero", "hora"]).agg({"troca": ["mean", "sum"]})
tcprev_sem = tcprev.groupby(["aero", "diaSemana"]).agg({"troca": ["mean", "sum"]})
tcprev_aero = tcprev.groupby(["aero"]).agg({"troca": ["mean", "sum"]})

In [57]:
tcreal = pd.read_csv("data/tc-real.csv")
tcreal["hora"] = pd.to_datetime(tcreal["hora"], unit='ms')
tcreal["diaSemana"] = tcreal["hora"].dt.dayofweek

tcreal['HoraData'] = tcreal['hora'].dt.strftime('%Y-%m-%d %H')
tcreal["hora"] = tcreal["hora"].dt.hour

In [56]:
#tcreal = pd.read_csv("data/tc-real.csv")
tcreal

,hora,nova_cabeceira,antiga_cabeceira,aero
0,1654092843000,32,03,FL
1,1654109470000,18,12,RF
2,1654110745000,16,13,CF
3,1654110758000,15,26,GL
4,1654110777000,15,26,GL
...,...,...,...,...
31694,1681588514000,15,30,KP
31695,1681589865000,17,10,SV
31696,1681590531000,10,17,SV
31697,1681590738000,15,08,GL


In [58]:
# Mesclar os dataframes usando a coluna "HoraData"

df_merge = pd.merge(bimtra, tcreal, on='HoraData', how='left')

# Preencher NaN na coluna de espera com 0 (sem espera)
df_merge['trocareal'] = 1
df_merge = df_merge.groupby("flightid").agg({"trocareal": "sum"})
bimtra = bimtra.merge(df_merge.reset_index(), on="flightid", how="left")

In [59]:
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,HoraData,esperas,troca
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,2022-06-01 00,0,3
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,2022-06-01 00,0,3
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,2022-06-01 00,0,3
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,2022-06-01 00,0,3
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,2022-06-01 01,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
278884,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,2023-04-15 12,0,1
278885,82ab2350af0301d57fd2041b7504f919,SBGR,SBRJ,2023-04-15 13:14:51,2023-04-15 13:54:44,2393,SBGRSBRJ,5,13,2023-04-15 13,0,6
278886,25fe7303ad56ca0f9976b8243546305b,SBSP,SBRJ,2023-04-15 17:11:23,2023-04-15 18:11:42,3619,SBSPSBRJ,5,17,2023-04-15 17,4,6
278887,d36199f6d9a3febc09113013e253739d,SBPA,SBRJ,2023-04-15 17:59:42,2023-04-15 19:46:02,6380,SBPASBRJ,5,17,2023-04-15 17,8,12


In [42]:

tcreal_hora = tcreal.groupby(["aero", "hora"]).agg({"aero": ["count"]})
tcreal_sem = tcreal.groupby(["aero", "diaSemana"]).agg({"aero": ["count"]})
tcreal_aero = tcreal.groupby(["aero"]).agg({"aero": ["count"]})

In [43]:
tcreal_aero

,aero
,count
aero,
BR,1153
CF,912
CT,1509
FL,6151
GL,2491
GR,814
KP,2060
PA,910


In [49]:
tcprev = pd.read_csv("data/tc-prev.csv")

In [46]:
bimtra["aero"] = bimtra["origem"].str[2:]

In [47]:
len(bimtra)

262416

In [50]:
tcprev_hora.reset_index(inplace=True)

In [53]:
tcprev_hora.columns = tcprev_hora.columns.droplevel(0)

In [56]:
tcprev_hora.columns = ["aero", "hora", "troca_prev_mean", "troca_prev_sum"]
tcprev_aero.reset_index(inplace=True)
tcprev_aero.columns = tcprev_aero.columns.droplevel(0)
tcprev_aero.columns = ["aero", "troca_prev_mean", "troca_prev_sum"]
tcprev_sem.reset_index(inplace=True)
tcprev_sem.columns = tcprev_sem.columns.droplevel(0)
tcprev_sem.columns = ["aero", "diaSemana", "troca_prev_mean", "troca_prev_sum"]
tcreal_hora.reset_index(inplace=True)
tcreal_hora.columns = tcreal_hora.columns.droplevel(0)
tcreal_hora.columns = ["aero", "hora", "troca_real"]
tcreal_aero.reset_index(inplace=True)
tcreal_aero.columns = tcreal_aero.columns.droplevel(0)
tcreal_aero.columns = ["aero", "troca_real"]
tcreal_sem.reset_index(inplace=True)
tcreal_sem.columns = tcreal_sem.columns.droplevel(0)
tcreal_sem.columns = ["aero", "diaSemana", "troca_real"]


In [58]:
bimtra.columns

Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr', 'target', 'linha',
       'diaSemana', 'hora', 'aero_x', 'mean_x', 'sum_x', 'aero_y', 'mean_y',
       'sum_y', 'aero', 'mean', 'sum'],
      dtype='object')

In [60]:
bimtra = bimtra.merge(tcprev_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcprev_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcprev_aero, on=["aero"], how="left")
bimtra = bimtra.merge(tcreal_hora, on=["aero", "hora"], how="left")
bimtra = bimtra.merge(tcreal_sem, on=["aero", "diaSemana"], how="left")
bimtra = bimtra.merge(tcreal_aero, on=["aero"], how="left")
bimtra

,flightid,origem,destino,dt_dep,dt_arr,target,linha,diaSemana,hora,aero_x,...,sum,troca_prev_mean_x,troca_prev_sum_x,troca_prev_mean_y,troca_prev_sum_y,troca_prev_mean,troca_prev_sum,troca_real_x,troca_real_y,troca_real
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,2022-06-01 00:44:57,2022-06-01 01:26:00,2463,SBKPSBRJ,2,0,SBKP,...,1492,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,2022-06-01 00:57:01,2022-06-01 01:32:53,2152,SBGRSBRJ,2,0,SBGR,...,1986,0.160494,52,0.293478,324,0.272119,2116,18.0,134,814
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,2022-06-01 00:26:52,2022-06-01 01:43:45,4613,SBBRSBRJ,2,0,SBBR,...,190,0.160494,52,0.293478,324,0.272119,2116,35.0,210,1153
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,2022-06-01 00:58:40,2022-06-01 01:41:41,2581,SBKPSBCF,2,0,SBKP,...,1492,0.160494,52,0.293478,324,0.272119,2116,23.0,288,2060
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,2022-06-01 01:00:38,2022-06-01 02:33:05,5547,SBCFSBFL,2,1,SBCF,...,1170,0.160494,52,0.293478,324,0.272119,2116,25.0,109,912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,SBGR,SBRJ,2023-04-15 21:41:11,2023-04-15 22:16:49,2138,SBGRSBRJ,5,21,SBGR,...,1986,0.259259,84,0.303442,335,0.272119,2116,85.0,96,814
262412,c1e1fcbfd1d1c67c01da384ad104c0df,SBBR,SBRJ,2023-04-15 21:07:10,2023-04-15 22:26:11,4741,SBBRSBRJ,5,21,SBBR,...,190,0.259259,84,0.303442,335,0.272119,2116,76.0,130,1153
262413,612af5b2b7cc48161bee87965785282b,SBFL,SBRJ,2023-04-15 21:30:27,2023-04-15 22:31:36,3669,SBFLSBRJ,5,21,SBFL,...,124,0.259259,84,0.303442,335,0.272119,2116,340.0,850,6151
262414,5efa9c4eb13e214f29a73a82cf7822e4,SBSV,SBRJ,2023-04-15 12:00:13,2023-04-15 14:00:05,7192,SBSVSBRJ,5,12,SBSV,...,72,0.351852,114,0.303442,335,0.272119,2116,590.0,1348,11140


In [61]:
train = bimtra.merge(resumo, right_on="linha_first", left_on="linha", how="left")

In [97]:
resumo

,linha_first,flightid_,dt_radar_<lambda>,dt_radar_<lambda>.1,dt_radar_<lambda>.2,dt_radar_<lambda>.3,lat_mean,lat_min,lat_max,lat_std,...,lon_max,lon_std,flightlevel_mean,flightlevel_min,flightlevel_max,flightlevel_std,speed_mean,speed_min,speed_max,speed_std
0,NaN,count,mean,min,max,std,mean,min,max,std,...,max,std,mean,min,max,std,mean,min,max,std
1,SBBRSBCF,2644,3210.6637670196674,0.0,8277.0,506.9815754584058,-0.3135663552228941,-0.4095989811,-0.2739212441,0.002530041610979571,...,-0.7026923893,0.0028508135030837728,223.23009635516672,1.0,410.0,15.414484379289918,365.208309469011,1.0,881.0,16.617049986812884
2,SBBRSBCT,959,5524.459854014599,4.0,8166.0,602.8147149739482,-0.36748848632055786,-0.4930038783,-0.1794724802,0.005855513262992156,...,-0.7082510707,0.0027255653742317646,281.071385902067,4.0,400.0,13.184672048521252,388.2540962465933,1.0,1555.0,13.50918093115723
3,SBBRSBFL,315,6614.263492063492,4.0,14041.0,824.4473638920381,-0.3867174708759314,-0.4862663929,-0.2765323694,0.006505166026804165,...,-0.7549999768,0.0017130580556180388,291.94466776671055,1.0,400.0,15.232888859932348,394.56650489239183,1.0,572.0,14.21473759918302
4,SBBRSBGL,639,4499.599374021909,362.0,7138.0,539.795847627226,-0.34150494094236883,-0.4146336416,-0.2324550936,0.004172015969809651,...,-0.6988433882,0.002983012702455903,259.66979013256446,1.0,607.0,13.252080132619195,393.2281877640942,1.0,552.0,15.93562582501301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,SBSVSBKP,1379,7368.569978245105,121.0,11225.0,761.98530512265,-0.31277198761030156,-0.4147787372,-0.0701902581,0.005460360215180889,...,-0.6646316611,0.004708078222442698,305.3446206649822,1.0,400.0,17.341259072228155,396.33274487402673,1.0,1286.0,16.681647582814314
134,SBSVSBPA,79,11771.683544303798,1683.0,13623.0,1455.2232848701617,-0.390669104767679,-0.525260494,-0.2250192926,0.010428194003739531,...,-0.6665442405,0.009512094269303203,324.43509942690906,1.0,400.0,8.228248797344037,392.19495205231044,34.0,490.0,8.362795726130809
135,SBSVSBRF,1491,3784.3105298457413,0.0,9182.0,526.7623314570509,-0.17761603549251634,-0.3543719597,-0.0650067621,0.002025703310561022,...,-0.6009507028,0.0013299560365280425,226.64509710011185,1.0,410.0,15.70704196991436,356.54213909988454,2.0,528.0,14.564714473730632
136,SBSVSBRJ,1630,6706.908588957055,843.0,10924.0,630.3015703620225,-0.3264385579605296,-0.5565484049,-0.1844531135,0.0038939621537456043,...,-0.6612747676,0.0027406425349698308,291.5118729723326,1.0,401.0,12.702016763699223,383.4791248139651,1.0,847.0,13.215119734561949


In [74]:
esperas_semana.to_csv("data/esperas_semana.csv", index=False)
esperas_hora.to_csv("data/esperas_hora.csv", index=False)

In [32]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def calcular_esparsidade(dataframe):
    # Converter o DataFrame em uma matriz esparsa
    matriz_esparsa = csr_matrix(dataframe.values)

    # Contar o número de elementos não nulos
    elementos_nao_nulos = matriz_esparsa.nnz

    # Calcular o número total de elementos na matriz
    total_elementos = matriz_esparsa.shape[0] * matriz_esparsa.shape[1]

    # Calcular a esparsidade
    esparsidade = 1 - (elementos_nao_nulos / total_elementos)

    return esparsidade

# Exemplo de uso

esparsidade = calcular_esparsidade(esperas)

print("Esparsidade do DataFrame:", esparsidade)


Esparsidade do DataFrame: 0.9540991814698712


In [31]:
esperas

esperas    
                      mean sum
aero hora                     
SBBR 1654041600000     0.0   0
     1654045200000     0.0   0
     1654048800000     0.0   0
     1654052400000     0.0   0
     1654056000000     0.0   0
...                    ...  ..
SBSV 1681585200000     0.0   0
     1681588800000     0.0   0
     1681592400000     0.0   0
     1681596000000     0.0   0
     1681599600000     0.0   0

[91872 rows x 2 columns]

In [62]:
train["origem"], _ = pd.factorize(train["origem"])
train["destino"], _ = pd.factorize(train["destino"])


In [63]:
train["linha"], _ = pd.factorize(train["linha"])


In [64]:
del train["linha_first"]


In [65]:
del train["dt_dep"], train["dt_arr"], train["aero_x"]

In [66]:
del train["aero_y"]

In [67]:
del train["aero"]

In [70]:
train

,flightid,origem,destino,target,linha,diaSemana,hora,mean_x,sum_x,mean_y,...,lon_max,lon_std,flightlevel_mean,flightlevel_min,flightlevel_max,flightlevel_std,speed_mean,speed_min,speed_max,speed_std
0,fcb2bf90345705318213ae1307c0f901,0,0,2463,0,2,0,0.694444,225,0.221014,...,-0.7024397788,0.0033911938063553512,155.34488648758892,1.0,410.0,10.830585205248747,322.5658956783692,1.0,855.0,16.348628987169015
1,c7c5c10716335b048f86d8c52fcba3f2,1,0,2152,1,2,0,0.987654,320,0.254529,...,-0.6689091105,0.004944606649096218,132.86727827511172,1.0,471.0,9.681570140183254,305.9386740015981,1.0,576.0,16.3842888246202
2,3162de7203a972f071d5a48e8f0f4828,2,0,4613,2,2,0,0.000000,0,0.034420,...,-0.7188257205,0.0030770205896807823,261.70366305291907,1.0,411.0,14.824040870084831,384.16901657635884,1.0,1365.0,17.765934928708287
3,8fcd243e9b9eebbe62ab145ce04b6ab5,0,1,2581,3,2,0,0.694444,225,0.221014,...,-0.6918451014,0.002714306069815275,196.2949962157983,1.0,410.0,15.784746321116943,352.7619864507692,1.0,917.0,17.81165593918845
4,504a62621cd231d6ab67e674ce538cd3,3,2,5547,4,2,1,0.055556,18,0.173007,...,-0.7650227937,0.0015262837761787745,260.85033157991717,1.0,400.0,14.632191848997632,365.45367138614193,1.0,519.0,13.370615410380863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262411,12d84d9dd7450b2cac83cdf041f58d67,1,0,2138,1,5,21,0.123457,40,0.167572,...,-0.6689091105,0.004944606649096218,132.86727827511172,1.0,471.0,9.681570140183254,305.9386740015981,1.0,576.0,16.3842888246202
262412,c1e1fcbfd1d1c67c01da384ad104c0df,2,0,4741,2,5,21,0.074074,24,0.004529,...,-0.7188257205,0.0030770205896807823,261.70366305291907,1.0,411.0,14.824040870084831,384.16901657635884,1.0,1365.0,17.765934928708287
262413,612af5b2b7cc48161bee87965785282b,6,0,3669,37,5,21,0.033951,11,0.012681,...,-0.6872124596,0.003327307351236129,241.22151471637108,1.0,411.0,12.110501024534415,380.0117151986321,4.0,589.0,15.306431860431022
262414,5efa9c4eb13e214f29a73a82cf7822e4,10,0,7192,38,5,12,0.000000,0,0.003623,...,-0.6612747676,0.0027406425349698308,291.5118729723326,1.0,401.0,12.702016763699223,383.4791248139651,1.0,847.0,13.215119734561949


In [71]:

train.columns = [re.sub('[\[\]<>\'"\'.,]', '', col) for col in train.columns]
train.to_csv("data/train_cat_tc_esperas.csv", index=False)